In [ ]:
renv::load(here::here())


In [ ]:
library(readr)
library(dplyr)
library(tidyr)
library(stringr)
library(magrittr)
library(padr)
library(tibble)
library(scales)
library(kableExtra)
library(purrr)
library(ggplot2)
library(ggthemr)
library(here)
library(lubridate)
library(animation)
ggthemr("fresh")
devtools::load_all()


In [ ]:
load(paste0(here(), "/data/other/species.RData"))
load(paste0(here(), "/data/other/stations.RData"))

In [ ]:

data_t2m <- import_data_dwh(paste0(here(), "/data/dwh/t2m_dwh_daily.txt"), parameter = "t2m") %>%
  # Maybe only retrieve latest 10 years for a more up-to-date picture
  # filter(date >= as.Date("2010-01-01")) %>%
  mutate(month = month(date), day = day(date)) %>%
  group_by(station, month, day) %>%
  summarise(t2m = round(mean(value), 4), .groups = "drop") %>%
  group_by(station) %>%
  mutate(weight = row_number(),
         t2m_weighted = weight * t2m) %>%
  ungroup()


In [ ]:
data_dwh <- import_data_dwh(paste0(here(), "/data/dwh/pollen_dwh_hourly.txt")) %>%
  aggregate_pollen() %>%
  impute_daily()

species_sel <- "alnu"
model_sel <- "pheno_v4"
year <- "21"
path <- paste0("/scratch/sadamov/wd/", year, "_", species_sel, "_", model_sel, "/")

data_sdes  <- import_data_cosmo(paste0(path, "mod_sdes_combined.txt"), type = "Phenology V4") %>%
  aggregate_pollen() %>%
  impute_daily()
data_saisn <- import_data_cosmo(paste0(path, "mod_saisn_combined.txt"), type = "Phenology V4") %>%
  aggregate_pollen() %>%
  impute_daily()
data_tthrs <- import_data_cosmo(paste0(path, "mod_tthrs_combined.txt"), type = "Phenology V4") %>%
  aggregate_pollen() %>%
  impute_daily()
data_ctsum <- import_data_cosmo(paste0(path, "mod_ctsum_combined.txt"), type = "Phenology V4") %>%
  aggregate_pollen() %>%
  impute_daily()

if (species_sel == "poac"){
data_saisl <- import_data_cosmo(paste0(path, "mod_saisl_combined.txt"), type = "Phenology V4") %>%
  aggregate_pollen() %>%
  impute_daily()
} else {
data_tthre <- import_data_cosmo(paste0(path, "mod_tthre_combined.txt"), type = "Phenology V4") %>%
  aggregate_pollen() %>%
  impute_daily()
}

invisible(
data_dwh %<>% filter(
  between(date, min(data_sdes$date), max(data_sdes$date)),
  taxon == unique(data_sdes$taxon)
))

data_pheno <- data_sdes %>%
  select(date, station, sdes = value) %>%
  left_join(data_saisn %>%
    select(date, station, saisn = value),
    by = c("date", "station")) %>%
  left_join(data_tthrs %>%
    select(date, station, tthrs = value),
    by = c("date", "station")) %>%
  left_join(data_ctsum %>%
    select(date, station, ctsum = value),
    by = c("date", "station"))

if (species_sel == "poac"){
  data_pheno %<>%
  left_join(data_saisl %>%
    select(date, station, saisl = value),
    by = c("date", "station"))
} else {
  data_pheno %<>%
  left_join(data_tthre %>%
    select(date, station, tthre = value),
    by = c("date", "station"))
}


data_pheno %<>%
  mutate(month = month(date), day = day(date)) %>%
  left_join(data_t2m, by = c("station", "month", "day")) %>%
  # In Wolfgang the season end is never reached (tthre too high?)
  filter(!station %in% c("Wolfgang", "La-Chaux-de-Fonds"))


In [ ]:
data_pheno %>%
filter(station == "Lugano")

In [ ]:
if (species_sel == "alnu") {
  curves_pheno <- data_pheno %>%
    split(.$station) %>%
    # Looping through the stations
    map(~ .x %>%
      # Looping through the days
      pmap(function(...) {
        current <- tibble(...)

        helper <- data_pheno %>%
          filter(station == current$station) %>%
          mutate(
            tthrs_dummy = case_when(
              date < current$date & current$ctsum < current$tthrs ~ 0,
              date == current$date & current$ctsum < current$tthrs ~ ctsum,
              date > current$date & current$ctsum < current$tthrs ~ t2m_weighted
            ),
            tthre_dummy = case_when(
              date < current$date ~ 0,
              date == current$date ~ ctsum,
              date > current$date ~ t2m_weighted
            ),
            # Vectorize the condition
            current_status = if_else(current$ctsum >= current$tthrs, TRUE, FALSE),
            t2m_threshold = if_else(current_status, ctsum, cumsum(tthrs_dummy)),
            in_season = if_else(t2m_threshold >= current$tthrs, 1, 0),
            day_in_season = cumsum(in_season),
            tthre_threshold = cumsum(tthre_dummy)
          )

        day_in_season <- helper %>%
          pull(day_in_season)

        if (current$ctsum >= current$tthre) {
          saisl <- helper %>%
            filter(ctsum >= current$tthre) %>%
            arrange(date) %>%
            slice(1) %>%
            pull(day_in_season)
        } else {
          saisl <- helper %>%
            filter(tthre_threshold >= current$tthre) %>%
            arrange(date) %>%
            slice(1) %>%
            pull(day_in_season)
        }

        # print(current$station)
        # print(current$date)
        # print(saisl)

        tibble(
          date = unique(data_pheno$date),
          value =
            ((1.16) / (1 + exp((-0.676 * 42.788 / saisl) *
              (day_in_season - 12.457 * saisl / 42.788)))) *
              ((1.16) / (1 + exp((0.193 * 42.788 / saisl) *
                (day_in_season - 23.435 * saisl / 42.788))))
        ) %>%
          mutate(value = rescale(value, to = c(0, 1)))
      }) %>%
      setNames(unique(data_pheno$date)))
}

In [ ]:
if (species_sel == "poac") {
  curves_pheno <- data_pheno %>%
    split(.$station) %>%
    # Looping through the stations
    map(~ .x %>%
      # Looping through the days
      pmap(function(...) {
        current <- tibble(...)

        day_in_season <- data_pheno %>%
          filter(station == current$station) %>%
          mutate(
            t2m_dummy = case_when(
              date < current$date & current$ctsum < current$tthrs ~ 0,
              date == current$date & current$ctsum < current$tthrs ~ ctsum,
              date > current$date & current$ctsum < current$tthrs ~ t2m_weighted
            ),
            # Vectorize the condition
            current_status = if_else(current$ctsum >= current$tthrs, TRUE, FALSE),
            t2m_threshold = if_else(current_status, ctsum, cumsum(t2m_dummy)),
            in_season = if_else(t2m_threshold >= current$tthrs, 1, 0),
            day_in_season = cumsum(in_season)
          ) %>%
          pull(day_in_season)

        tibble(
          date = unique(data_pheno$date),
          value =
            ((0.993 + 0.02) / (1 + exp((-0.237 * 73.651 / current$saisl) *
              (day_in_season - 18.960 * current$saisl / 73.651)))) *
              ((0.993 + 0.02) / (1 + exp((0.152 * 73.651 / current$saisl) *
                (day_in_season - 60.502 * current$saisl / 73.651))))
        ) %>%
          mutate(value = rescale(value, to = c(0, 1)))
      }) %>%
      setNames(unique(data_pheno$date)))
}

In [ ]:

dates <- unique(data_pheno$date)
station_sel <- "Lugano"

plot_collection <- map2(dates, seq_along(dates), function(first, second) {
  data_dwh %>%
    filter(station == station_sel) %>%
    mutate(value = rescale(value, to = c(0, 1))) %>%
    filter(date <= first) %>%
    ggplot() +
    ylim(c(0, 1)) +
    scale_x_date(
      date_breaks = "1 month",
      date_labels = "%B",
      limits = c(min(data_pheno$date), max(data_pheno$date))
    ) +
    ylab("") +
    xlab(min(year(data_pheno$date))) +
    ggtitle("Season Description in Cosmo - Phenology Curve Update") +
    geom_col(aes(x = date, y = value), col = "#1ba9ade7", fill = "#1ba9ade7") +
    geom_line(
      data = curves_pheno[[station_sel]][[as.character(first)]],
      aes(x = date, y = value),
      col = "#f5c04e"
    ) +
    geom_line(
      data = curves_pheno[[station_sel]][[as.character(min(dates))]],
      aes(x = date, y = value),
      col = "#f15236"
    ) +
    geom_point(
      data = data_pheno %>%
        filter(station == station_sel, date <= first),
      aes(x = date, y = sdes),
      col = "#6b4900",
      shape = 8
    )
})


In [ ]:
saveGIF(
  {
    for (i in seq_along(plot_collection)) {
      print(plot_collection[[i]])
    }
  },
  movie.name = paste0(here(), "/vignettes/figures/Phenology.gif"),
  interval = 1/25,
  ani.width = 720,
  ani.height = 480
)